## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
data_text=pd.read_csv('/content/gdrive/MyDrive/abcnews-date-text.csv')

Mounted at /content/gdrive


In [2]:
documents=pd.DataFrame(data_text)
documents.head

<bound method NDFrame.head of          publish_date                                      headline_text
0            20030219  aba decides against community broadcasting lic...
1            20030219     act fire witnesses must be aware of defamation
2            20030219     a g calls for infrastructure protection summit
3            20030219           air nz staff in aust strike for pay rise
4            20030219      air nz strike to affect australian travellers
...               ...                                                ...
1103660      20171231  the ashes smiths warners near miss liven up bo...
1103661      20171231            timelapse: brisbanes new year fireworks
1103662      20171231           what 2017 meant to the kids of australia
1103663      20171231   what the papodopoulos meeting may mean for ausus
1103664      20171231  who is george papadopoulos the former trump ca...

[1103665 rows x 2 columns]>

In [3]:
documents.drop('publish_date',axis=1,inplace=True)

In [4]:
documents.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [5]:
documents['index']=documents.index

In [6]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


Let's glance at the dataset:

In [7]:
'''
print the total number of documents
'''
print(documents.shape)

(1103665, 2)


In [8]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [9]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [10]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [11]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [12]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [13]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
lemmatizer=WordNetLemmatizer()
stemmer=PorterStemmer()
nltk.download('stopwords')
def lemmatize_stemming(text):

  word=text.lower()
  word=nltk.word_tokenize(word)
  word=[lemmatizer.lemmatize(w) for w in word]
  word=[stemmer.stem(l) for l in word]
  word=' '.join(word)
  return(word)
  pass

#lemmatize_stemming(documents.iloc[0:5,0:])

# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
            re=lemmatize_stemming(token)
            result.append(re)
            pass
        
        
    return result

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
'''
Preview a document after preprocessing
'''
document_num = 4298
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['pontings', 'new', 'ball', 'warning']


Tokenized and lemmatized document: 
['pont', 'ball', 'warn']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [15]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs=documents['headline_text'].map(preprocess)

In [16]:
'''
Preview 'processed_docs'
'''


processed_docs[:10]




0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [17]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
from gensim.corpora import Dictionary
#  Todo

#dct=Dictionary(processed_docs)


In [18]:
dictionary=gensim.corpora.dictionary.Dictionary(documents=processed_docs, prune_at=2000000)

In [19]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [20]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [21]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
#  Todo:
bow_corpus = [dictionary.doc2bow(document) for document in processed_docs]


In [22]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(346, 1), (2717, 1), (4282, 1)]

In [23]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 346 ("warn") appears 1 time.
Word 2717 ("pont") appears 1 time.
Word 4282 ("ball") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [24]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
# TODO
tf_idf = models.TfidfModel(bow_corpus)

In [25]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tf_idf[bow_corpus]

In [26]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903416018683463),
 (1, 0.385232906171445),
 (2, 0.4974398602008866),
 (3, 0.5056144643792737)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [27]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
# TODO
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)


In [28]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.034*"australia" + 0.028*"queensland" + 0.021*"donald" + 0.018*"elect" + 0.016*"market" + 0.016*"world" + 0.013*"open" + 0.012*"share" + 0.011*"bank" + 0.010*"year"


Topic: 1 
Words: 0.021*"nation" + 0.014*"chang" + 0.014*"live" + 0.014*"council" + 0.012*"fight" + 0.012*"commun" + 0.012*"price" + 0.012*"claim" + 0.012*"plan" + 0.011*"say"


Topic: 2 
Words: 0.048*"polic" + 0.030*"sydney" + 0.028*"charg" + 0.022*"murder" + 0.019*"court" + 0.018*"face" + 0.017*"brisban" + 0.016*"death" + 0.014*"accus" + 0.014*"miss"


Topic: 3 
Words: 0.021*"canberra" + 0.017*"turnbul" + 0.012*"australia" + 0.012*"final" + 0.012*"street" + 0.010*"game" + 0.009*"world" + 0.009*"melbourn" + 0.009*"race" + 0.009*"star"


Topic: 4 
Words: 0.024*"adelaid" + 0.022*"perth" + 0.017*"state" + 0.015*"peopl" + 0.013*"student" + 0.012*"countri" + 0.012*"citi" + 0.011*"home" + 0.011*"darwin" + 0.010*"prison"


Topic: 5 
Words: 0.021*"school" + 0.017*"life" + 0.016*"health" + 0.012*"meet" + 0.012*"m

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [29]:
'''
Define lda model using corpus_tfidf
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

In [30]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.028*"elect" + 0.024*"say" + 0.015*"power" + 0.014*"labor" + 0.012*"marriag" + 0.011*"polit" + 0.011*"senat" + 0.010*"vote" + 0.009*"liber" + 0.009*"parti"


Topic: 1 Word: 0.057*"australia" + 0.028*"queensland" + 0.020*"school" + 0.015*"tasmanian" + 0.015*"indigen" + 0.014*"live" + 0.012*"commun" + 0.011*"chang" + 0.009*"victorian" + 0.009*"industri"


Topic: 2 Word: 0.028*"woman" + 0.028*"charg" + 0.028*"court" + 0.023*"child" + 0.021*"murder" + 0.017*"face" + 0.015*"state" + 0.015*"jail" + 0.014*"polic" + 0.014*"accus"


Topic: 3 Word: 0.051*"trump" + 0.014*"say" + 0.013*"deal" + 0.013*"korea" + 0.012*"protest" + 0.011*"worker" + 0.011*"talk" + 0.009*"fear" + 0.009*"save" + 0.008*"right"


Topic: 4 Word: 0.016*"health" + 0.015*"south" + 0.015*"tasmania" + 0.014*"council" + 0.014*"fund" + 0.013*"leagu" + 0.013*"water" + 0.013*"servic" + 0.012*"plan" + 0.011*"concern"


Topic: 5 Word: 0.023*"adelaid" + 0.019*"canberra" + 0.016*"final" + 0.012*"island" + 0.011*"citi" + 

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [31]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [32]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.5249999761581421	 
Topic: 0.021*"canberra" + 0.017*"turnbul" + 0.012*"australia" + 0.012*"final" + 0.012*"street" + 0.010*"game" + 0.009*"world" + 0.009*"melbourn" + 0.009*"race" + 0.009*"star"

Score: 0.27497681975364685	 
Topic: 0.021*"school" + 0.017*"life" + 0.016*"health" + 0.012*"meet" + 0.012*"minist" + 0.011*"concern" + 0.011*"road" + 0.011*"fund" + 0.010*"indigen" + 0.010*"parti"

Score: 0.02500789798796177	 
Topic: 0.025*"north" + 0.022*"south" + 0.013*"island" + 0.013*"west" + 0.013*"china" + 0.013*"say" + 0.013*"korea" + 0.012*"polit" + 0.012*"power" + 0.011*"busi"

Score: 0.025005698204040527	 
Topic: 0.049*"australian" + 0.034*"govern" + 0.016*"tasmania" + 0.013*"victoria" + 0.011*"farmer" + 0.010*"water" + 0.010*"rural" + 0.010*"storm" + 0.010*"refuge" + 0.009*"wall"

Score: 0.02500537969172001	 
Topic: 0.024*"adelaid" + 0.022*"perth" + 0.017*"state" + 0.015*"peopl" + 0.013*"student" + 0.012*"countri" + 0.012*"citi" + 0.011*"home" + 0.011*"darwin" + 0.010*"pris

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [33]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
# Todo
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.27500009536743164	 
Topic: 0.016*"health" + 0.015*"south" + 0.015*"tasmania" + 0.014*"council" + 0.014*"fund" + 0.013*"leagu" + 0.013*"water" + 0.013*"servic" + 0.012*"plan" + 0.011*"concern"

Score: 0.27500006556510925	 
Topic: 0.019*"warn" + 0.017*"turnbul" + 0.015*"miss" + 0.012*"victoria" + 0.012*"share" + 0.012*"countri" + 0.012*"street" + 0.011*"rate" + 0.011*"china" + 0.011*"busi"

Score: 0.27499982714653015	 
Topic: 0.028*"elect" + 0.024*"say" + 0.015*"power" + 0.014*"labor" + 0.012*"marriag" + 0.011*"polit" + 0.011*"senat" + 0.010*"vote" + 0.009*"liber" + 0.009*"parti"

Score: 0.02500000037252903	 
Topic: 0.057*"australia" + 0.028*"queensland" + 0.020*"school" + 0.015*"tasmanian" + 0.015*"indigen" + 0.014*"live" + 0.012*"commun" + 0.011*"chang" + 0.009*"victorian" + 0.009*"industri"

Score: 0.02500000037252903	 
Topic: 0.028*"woman" + 0.028*"charg" + 0.028*"court" + 0.023*"child" + 0.021*"murder" + 0.017*"face" + 0.015*"state" + 0.015*"jail" + 0.014*"polic" + 0.014*"

## Step 6: Testing model on unseen document ##

In [34]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6200000047683716	 Topic: 0.021*"canberra" + 0.017*"turnbul" + 0.012*"australia" + 0.012*"final" + 0.012*"street"
Score: 0.2199999839067459	 Topic: 0.020*"coast" + 0.015*"time" + 0.013*"deal" + 0.013*"marriag" + 0.013*"gold"
Score: 0.019999999552965164	 Topic: 0.034*"australia" + 0.028*"queensland" + 0.021*"donald" + 0.018*"elect" + 0.016*"market"
Score: 0.019999999552965164	 Topic: 0.021*"nation" + 0.014*"chang" + 0.014*"live" + 0.014*"council" + 0.012*"fight"
Score: 0.019999999552965164	 Topic: 0.048*"polic" + 0.030*"sydney" + 0.028*"charg" + 0.022*"murder" + 0.019*"court"
Score: 0.019999999552965164	 Topic: 0.024*"adelaid" + 0.022*"perth" + 0.017*"state" + 0.015*"peopl" + 0.013*"student"
Score: 0.019999999552965164	 Topic: 0.021*"school" + 0.017*"life" + 0.016*"health" + 0.012*"meet" + 0.012*"minist"
Score: 0.019999999552965164	 Topic: 0.025*"north" + 0.022*"south" + 0.013*"island" + 0.013*"west" + 0.013*"china"
Score: 0.019999999552965164	 Topic: 0.049*"australian" + 0.034*

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Double click here to append points

- 